In [ ]:
import cv2
import numpy as np

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0
# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = './Users/myfaces1/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

In [ ]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

data_path ="./Users/myfaces1/"
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

Training_Data, Labels = [], []

for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)
    

    
Labels = np.asarray(Labels, dtype = np.int32)

model = cv2.face_LBPHFaceRecognizer.create()
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model Trained Successfully")




In [ ]:
import cv2
import numpy as np
import os
import pywhatkit
from datetime import datetime
import smtplib
import subprocess as sp
import time

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def email():
    server = smtplib.SMTP_SSL("smtp.gmail.com",465)
    print("Server Start")
    server.login("email address","password")
    print("login Complete")
    server.sendmail("Charushila","charushilabhadane123@gmail.com" , "this is face of Charushila")
    
    server.quit()
    print("mail send")
    
def whatsApp():  
    now = datetime.now()
    hours = int(now.strftime("%H"))
    minutes = int(now.strftime("%M")) + 1
    pywhatkit.sendwhatmsg('Mobile no','Hi Charushila Your face is recognised',hours,minutes)
    print("whatsApp message send")

def Instance_EBS():
    #Creating an instance
    instance_id = sp.getoutput("aws ec2 run-instances --image-id  ami-011c99152163a87ae --count 1 --instance-type t2.micro --key-name AWSkey --security-group-ids sg-61c4031d --subnet-id subnet-0fb84c64")
    print("Instance Id : "+str(instance_id))
    #Creating a volume
    volume_id = sp.getoutput("aws ec2 create-volume --volume-type gp2 --size 5 --availability-zone ap-south-1a")
    print("Volume Id : "+str(volume_id) )
    time.sleep(20)
    #Attaching instance to volume
    sp.getoutput("aws ec2 attach-volume --volume-id vol-02aeff71c4a3d2b9a --instance-id i-09e04db71d14e0d16 --device /dev/sdf".format(volume_id,instance_id))
    print("Volume and Instance Attached") 
    
def face_detector(img, size=0.5):
                               
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
        
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        img1 = img[y:y+h, x:x+w]
        img1 = cv2.resize(img1, (200, 200))
    return img, img1

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()    
    image, face = face_detector(frame)
   
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        results = model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence > 70:
            cv2.putText(image, "Hey Charushila", (250, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )  
            email()
            whatsApp()
            break     
        elif confidence < 70 :
            cv2.putText(image, "Hey You are another", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )
            Instance_EBS() 
            break

        
        else:
            cv2.putText(image, "I dont know, who r you", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        
        pass
        
    if cv2.waitKey(1) == 13: 
        break
        
cap.release()
cv2.destroyAllWindows()

    

